In [2]:
import numpy as np
import cv2
import os
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.utils import load_img
from tensorflow.keras import layers, models
import pathlib
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
folder_path = "C:/Users/hp/images"
#img = load_img("C:/Users/hp/images/train/fear" + "/" + os.listdir("C:/Users/hp/images/train/fear")[0])
#plt.imshow(img)

data_dir = pathlib.WindowsPath(folder_path)
data_dir

WindowsPath('C:/Users/hp/images')

In [4]:
def generate_dataset(Type):
    Emotion_dict = {
        "angry": list(data_dir.glob(Type + '/angry/*.jpg'))[0:1000],
        "disgust": list(data_dir.glob(Type + '/disgust/*.jpg')),
        "fear": list(data_dir.glob(Type + '/fear/*.jpg'))[0:1000],
        "happy": list(data_dir.glob(Type + '/happy/*.jpg'))[0:1000],
        "neutral": list(data_dir.glob(Type + '/neutral/*.jpg'))[0:1000],
        "sad": list(data_dir.glob(Type + '/sad/*.jpg'))[0:1000],
        "surprise": list(data_dir.glob(Type + '/surprise/*.jpg'))[0:1000],
    }

    Emotion_label = {
        "angry": 0,
        "disgust": 1,
        "fear": 2,
        "happy": 3,
        "neutral": 4,
        "sad": 5,
        "surprise": 6,
    }
    X, y = [], []

    for emotion, images in Emotion_dict.items():
        for image in images:
            img = cv2.imread(str(image))
            resized_img = cv2.resize(img, (224, 224))
            X.append(resized_img)
            y.append(Emotion_label[emotion])

    X = np.array(X)
    y = np.array(y)

    return X, y

In [5]:
X_train, y_train = generate_dataset("train")
X_test, y_test = generate_dataset("validation")

In [6]:
X_train = X_train / 255.0
X_test = X_test/255.0

In [6]:
classifier = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
                   input_shape = (224, 224, 3),
                   trainable = False)
])

classifier.build([None, 224, 224, 3])


(224, 224, 3)

In [12]:
model = keras.models.Sequential([
    layers.Conv2D(filters = 80,kernel_size = (3,3), activation = 'relu',input_shape = (224,224,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),
    layers.Dense(100, activation = 'relu'),
    layers.Dropout(0.5),
    layers.Dense(40, activation = 'relu'),
    layers.Dropout(0.3),
    layers.Dense(7, activation = 'softmax'),
])


In [13]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [14]:
tb_callback = tf.keras.callbacks.TensorBoard (
    log_dir = "logs/minor_model_9",
    histogram_freq = 2,
)

In [ ]:
model.fit( X_train, y_train, epochs=4, batch_size = 10, callbacks = [tb_callback])

Epoch 1/4
589/644 [==========================>...] - ETA: 43s - loss: 1.9616 - accuracy: 0.1567

In [ ]:
model.save('black_box_6.h5')

In [ ]:
def accuracy(A):
    count = 0
    y_t = y_test[4000:5500]
    for i in range(0, A.size):
        if A[i] == y_t[i]:
            count += 1
    acc = (count/A.size*100)
    return acc

In [ ]:
y_pre = model.predict(X_test[4000:5000])


In [ ]:
y_predict = [np.argmax(i) for i in y_pre]
y_predicted = np.array(y_predict)
print(accuracy(y_predicted))